In [16]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [17]:
#-----------HYPERPARAMETERS----------------#

SEQUENCE_LENGTH = [2, 4, 8, 16, 32]
HIDDEN_LENGTH = 20
N_FEATURES = 1

LEARNING_RATE = 1e-3
EPOCHS = 100
CUMSUM = True
NUM_OF_LAYERS = 3

In [18]:
#-----------Loading the dataset----------------#

zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

In [19]:
df = pd.read_csv(csv_path)
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
p (mbar),420551.0,989.212776,8.358481,913.60,984.20,989.58,994.72,1015.35
T (degC),420551.0,9.450147,8.423365,-23.01,3.36,9.42,15.47,37.28
Tpot (K),420551.0,283.492743,8.504471,250.60,277.43,283.47,289.53,311.34
Tdew (degC),420551.0,4.955854,6.730674,-25.01,0.24,5.22,10.07,23.11
rh (%),420551.0,76.008259,16.476175,12.95,65.21,79.30,89.40,100.00
VPmax (mbar),420551.0,13.576251,7.739020,0.95,7.78,11.82,17.60,63.77
VPact (mbar),420551.0,9.533756,4.184164,0.79,6.21,8.86,12.35,28.32
VPdef (mbar),420551.0,4.042412,4.896851,0.00,0.87,2.19,5.30,46.01
sh (g/kg),420551.0,6.022408,2.656139,0.50,3.92,5.59,7.80,18.13
H2OC (mmol/mol),420551.0,9.640223,4.235395,0.80,6.29,8.96,12.49,28.82


In [20]:
data = df['T (degC)'].to_numpy()

In [21]:
def create_weather_dataset_for_one_timetep_prediction(data, n_samples, sequence_length, n_features = 1, validation = False):
    """
    Function to create a dataset for a prediction task to predict the next value after the input sequence
    
    :param data: 
    :return: 
    """
    
    sequences = []
    labels = []
    
    for _ in range (n_samples):
        index = np.random.randint(0, len(data) - sequence_length -1)
        x = data[index:index+sequence_length]
        x = np.expand_dims(x, axis=-1)
        sequences.append(x)
        y = data[index+sequence_length+1]
        y = np.expand_dims(y, axis=-1)
        labels.append(y)
        
    if not validation:
        return tf.data.Dataset.from_tensor_slices((sequences, labels)).shuffle(n_samples).batch(32).prefetch(tf.data.AUTOTUNE)
    else:
        return tf.data.Dataset.from_tensor_slices((sequences, labels)).shuffle(n_samples).batch(n_samples).prefetch(tf.data.AUTOTUNE)

def create_weather_dataset_for_multiple_timesteps_prediction(data, n_samples, sequence_length, n_features = 1, validation = False):
    """
    Function to create a dataset for a prediction task to predict the next value after the input sequence
    
    :param data: 
    :return: 
    """
    
    sequences = []
    labels = []
    
    for _ in range (n_samples):
        index = np.random.randint(0, len(data) - sequence_length - sequence_length)
        x = data[index:index+sequence_length]
        x = np.expand_dims(x, axis=-1)
        sequences.append(x)
        
        y = data[index+sequence_length:index+sequence_length+sequence_length]
        y = np.expand_dims(y, axis=-1)
        labels.append(y)
    if not validation:
        return tf.data.Dataset.from_tensor_slices((sequences, labels)).shuffle(n_samples).batch(32).prefetch(tf.data.AUTOTUNE)
    else:
        return tf.data.Dataset.from_tensor_slices((sequences, labels)).shuffle(n_samples).batch(n_samples).prefetch(tf.data.AUTOTUNE)

In [22]:
def plot_predictions(input_sequence, expected_output, actual_output):
    plt.figure(figsize=(10, 6))
    plt.plot(input_sequence, label='Input Sequence', marker='o')
    plt.plot(range(len(input_sequence), len(input_sequence) + len(expected_output)),
             expected_output, label='Expected Output', marker='o')
    plt.plot(range(len(input_sequence), len(input_sequence) + len(actual_output)),
             actual_output[0], label='Actual Output', marker='o')
    plt.xlabel('Time Steps')
    plt.ylabel('Value')
    plt.title('Input Sequence, Expected Output, and Actual Output')
    plt.legend()
    plt.grid(True)
    
    # Festlegen der Achsengröße
    plt.xlim(0, len(input_sequence) + len(expected_output))
    plt.ylim(-20, 30)  # Beispiel: Festlegen der y-Achsen-Grenzen auf 0 und 100
    
    plt.show()

In [53]:
import tensorflow as tf
import tensorflow as tf
import tqdm
from tensorflow.python.ops.ragged import ragged_tensor
from tensorflow.python.ops import array_ops_stack

dataset = create_weather_dataset_for_one_timetep_prediction(data, n_samples=100, sequence_length=15)

def sliding_window(data, width: int, axis: int=-1, name=None):
    data = ragged_tensor.convert_to_tensor_or_ragged_tensor(data, name="data")

    slices = []
    for start in range(width):
      stop = None if start - width + 1 == 0 else start - width + 1
      if axis >= 0:
        idx = [slice(None)] * axis + [slice(start, stop)]
      else:
        idx = [Ellipsis, slice(start, stop)] + [slice(None)] * (-axis - 1)
      slices.append(data[idx])

    # Stack the slices.
    stack_axis = axis + 1 if axis >= 0 else axis
    return array_ops_stack.stack(slices, stack_axis)


class EmbeddTokenAndPosLayer(tf.keras.layers.Layer):
     
    def __init__(self, embedding_size, max_input_seq_len):
        super(EmbeddTokenAndPosLayer, self).__init__()

        self.embedding_size = embedding_size
        self.max_input_seq_len = max_input_seq_len
        self.embedding_pos = tf.keras.layers.Embedding(self.max_input_seq_len, self.embedding_size)

    def call(self, x):
        pos_embedd = self.embedding_pos( tf.range(self.max_input_seq_len) )

        return x + pos_embedd

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embedding_size):
        super(TransformerBlock, self).__init__()

        self.multi_head_attention = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=embedding_size)
        self.dropout1 = tf.keras.layers.Dropout(0.1)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(embedding_size, activation=None)
        self.dropout2 = tf.keras.layers.Dropout(0.1)

        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def call(self, x):
        out1 = self.multi_head_attention(x, x)
        out1 = self.dropout1(out1)

        in_out = self.layernorm1(out1 + x)

        out2 = self.dense1(in_out)
        out2 = self.dense2(out2)
        out2 = self.dropout2(out2)

        out2 = self.layernorm2(out2 + in_out)
        return out2

class TokenPredictor(tf.keras.Model):
    def __init__(self, vocabulary_size, embedding_size, max_input_seq_len):
        super(TokenPredictor, self).__init__()
        
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.max_input_seq_len = max_input_seq_len - 1

        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        self.loss_function = tf.keras.losses.MeanSquaredError()

        self.metrics_list = [
                        tf.keras.metrics.Mean(name="loss"),
                        tf.keras.metrics.Accuracy(name="acc"),
                        tf.keras.metrics.MeanSquaredError("mse") 
                       ]

        self.layer_list = [
            EmbeddTokenAndPosLayer(embedding_size, max_input_seq_len),
            TransformerBlock(embedding_size),
            tf.keras.layers.GlobalAveragePooling1D(),
            tf.keras.layers.Dense(vocabulary_size, activation=None)
        ]

    @tf.function # äußere
    def call(self, x, training=False):
        for layer in self.layer_list:
            try:
                x = layer(x,training)
            except:
                x = layer(x)
       
        return x
    
    @property
    def metrics(self):
        return self.metrics_list

    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_state()
    
    @tf.function
    def train_step(self, input_seq, target_token):
        
        with tf.GradientTape() as tape:
            predictions = self(input_seq, training=True)
            target_token = tf.expand_dims(target_token, -1)
            loss = self.loss_function(target_token, predictions) #+ tf.reduce_sum(self.losses)
        
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        # update loss metric
        self.metrics[0].update_state(loss)
        
        # for all metrics except loss, update states (accuracy etc.)
        for metric in self.metrics[1:]:
            metric.update_state(target_token,predictions)

        # Return a dictionary mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    def predict(self, input, k_top):
        
        num_toPredict = self.max_input_seq_len - input.shape[0] + 1
        if num_toPredict <= 0:
            print("Error: Text to long")
            return input

        for i in range(num_toPredict):

            # add padding
            padding_len = self.max_input_seq_len - input.shape[0] + 1
            padding_token_id = self.tokenizer.string_to_id("")
            padding = tf.zeros(shape=padding_len, dtype=tf.int32) + padding_token_id 

            ids_padded = tf.concat([input, padding] , axis=-1)

            # add batch dim
            ids_padded = tf.expand_dims(ids_padded, axis=0)

            prediction_ids = self(ids_padded)

            values, indices = tf.math.top_k(prediction_ids, k=k_top, sorted=True)

            idx = tf.random.categorical(values, num_samples=1)

            idx = tf.squeeze(idx) 
            indices = tf.squeeze(indices) 
            predicted_id = indices[idx]

    
            # append predicted token
            predicted_id = tf.cast(predicted_id, tf.int32)
            predicted_id = tf.expand_dims(predicted_id, axis=-1) # add 1st dim
            ids = tf.concat([ids, predicted_id], axis=-1)
        
        return ids
    

    def test_step(self, dataset):

        input_seq, target_token = dataset
        target_token = tf.expand_dims(target_token, -1)

        predictions = self(input_seq, training=False)
        
        loss = self.loss_function(target_token, predictions) #+ tf.reduce_sum(self.losses)
        
        self.metrics[0].update_state(loss)
        
        for metric in self.metrics[1:]:
            metric.update_state(target_token, predictions)

        return {m.name: m.result() for m in self.metrics}

def main():
    
    vocab_size = 6000
    embedding_size = 64
    max_seq_len = 15

    # Logging
    file_path = "test_logs/test" 
    train_summary_writer = tf.summary.create_file_writer(file_path)
    
    ids_dataset = dataset

    tokenPredictor = TokenPredictor(vocab_size, embedding_size, max_seq_len)

    log(train_summary_writer, tokenPredictor, ids_dataset, 0)

    NUM_EPOCHS = 600
    for epoch in range(NUM_EPOCHS):
        
        print(f"Epoch {epoch}:")

        for input_seq, target_token in tqdm.tqdm(ids_dataset,position=0, leave=True):
            metrics = tokenPredictor.train_step(input_seq, target_token)

        # print the metrics
        print([f"{key}: {value}" for (key, value) in zip(list(metrics.keys()), list(metrics.values()))])

        log(train_summary_writer, tokenPredictor, ids_dataset, epoch + 1)

        # Save weights
        if epoch % 100 == 0:
            tokenPredictor.save_weights(f"trainied_weights_epoch_{epoch}.weights.h5")
 

def log(train_summary_writer, tokenPredictor, dataset, epoch):

    with train_summary_writer.as_default():
        
        # Log metrics
        if epoch == 0:
            for data in dataset.take(1000): # approx full train dataset
                metrics = tokenPredictor.test_step(data)

        for metric in tokenPredictor.metrics:
            tf.summary.scalar(f"{metric.name}_train", metric.result(), step=epoch)

        tokenPredictor.reset_metrics()

main()

2024-03-31 10:20:23.539425: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 0:


100%|██████████| 4/4 [00:01<00:00,  2.78it/s]


['loss: 214.9613800048828', 'acc: 0.0', 'mse: 189.81964111328125']
Epoch 1:


100%|██████████| 4/4 [00:00<00:00, 149.06it/s]


...

['loss: 18.01059341430664', 'acc: 0.0', 'mse: 21.816221237182617']
Epoch 119:


100%|██████████| 4/4 [00:00<00:00, 111.15it/s]


['loss: 21.308841705322266', 'acc: 0.0', 'mse: 23.162321090698242']
Epoch 120:


 75%|███████▌  | 3/4 [00:00<00:00, 76.31it/s]


KeyboardInterrupt: 